# Main

> FrankenUI Tasks Example

In [2]:
#| default_exp main

## Imports

In [3]:
#| export
from fasthtml.common import *
import fasthtml.common as fh
from fh_frankenui.components import *
from fh_frankenui.core import *
import re
from fasthtml.components import Uk_theme_switcher
from utils import hjs

In [4]:
#| hide
#| eval: false
from utils import create_server
from fasthtml.jupyter import *

In [5]:
#| export
app,rt = fast_app(pico=False, hdrs=(*Theme.blue.headers(),*hjs))

In [7]:
#| hide
#| eval: false
server, Show = create_server(app)

In [ ]:
#| export
def with_layout(active):
    def decorator(func):
        def wrapper():
            original_content = func()
            return Body(cls="bg-background text-foreground")(
                Div(cls="flex flex-col md:flex-row w-full")(
                    Button(UkIcon("menu",50,50,cls='mt-4'), cls="md:hidden mb-4", uk_toggle="target: #mobile-sidebar"),
                    Div(sidebar(active), id='mobile-sidebar', hidden=True),
                    
                    Div(cls="md:flex w-full")(
                        Div(sidebar(active), cls="hidden md:block w-1/5"),
                        Div(original_content, cls='md:w-4/5 w-full mr-5', id="mobile-sidebar")
                    )
                ),
            )
        wrapper.__name__ = func.__name__
        return wrapper
    return decorator

## Examples

In [9]:
#| export
from tasks import tasks_homepage
from cards import cards_homepage
from dashboard import dashboard_homepage
from forms import forms_homepage
from music import music_homepage
from auth import auth_homepage
from playground import playground_homepage
from mail import mail_homepage

In [10]:
#| export
@rt
@with_layout('task')
def tasks():      return tasks_homepage
@rt
@with_layout('card')
def cards():      return cards_homepage
@rt
@with_layout('dashboard')
def dashboard():  return dashboard_homepage
@rt 
@with_layout('form')
def forms():      return forms_homepage
@rt
@with_layout('music')
def music():      return music_homepage
@rt
@with_layout('auth')
def auth():       return auth_homepage
@rt
@with_layout('playground')
def playground(): return playground_homepage
@rt
@with_layout('mail')
def mail():       return mail_homepage

## Getting Started

In [11]:
#| export
@rt
@with_layout('getting_started')
def getting_started():
    return Container(render_md(open('GettingStarted.md').read()))

## API Reference

In [12]:
#| export
import api_reference

In [ ]:
#| export
reference_fns = L([o for o in dir(api_reference) if o.startswith('docs_')])

In [13]:
reference_fns[:3]

(#3) ['docs_button_link','docs_cards','docs_containers']

In [105]:
#| export
def fnname2title(ref_fn_name): return ref_fn_name[5:].replace('_',' | ').title() 

In [14]:
#| export
api_ref_rts = [(f"/{o}", rt(f"/{o}")(with_layout(fnname2title(o))(getattr(api_reference, o)))) for o in reference_fns]

## Tutorials

## Theme

In [107]:
#| export
@rt
@with_layout('theme')
def themeswitcher(): return Div(Uk_theme_switcher(),cls="p-12")

## LLM Contexts

In [11]:
#| export
@rt
@with_layout('llms')
def llms():
    return Container(render_md(open('LLM Contexts.md').read()))

## Page Layout

In [ ]:
#| export
def sidebar(active=''):
    def create_li(title, href):
        is_active = title.lower() == active.lower()
        return Li(A(title, href=href), cls="uk-active" if is_active else "")

    return NavContainer(
        create_li("Getting Started", getting_started),
        create_li("LLMs", llms),
        NavParentLi(
            A(FullySpacedDiv("API Reference")),
            NavContainer(
                *[create_li(fnname2title(o), f"/{o}") for o in reference_fns],
                parent=False
            ),
        ),
        NavParentLi(
            A(FullySpacedDiv('Examples')),
            NavContainer(
                *[create_li(title, href) for title, href in [
                    ('Task', tasks),
                    ('Card', cards),
                    ('Dashboard', dashboard),
                    ('Form', forms),
                    ('Music', music),
                    ('Auth', auth),
                    ('Playground', playground),
                    ('Mail', mail),
                ]],
                parent=False
            ),
        ),
        create_li("Theme", themeswitcher),
        uk_nav=False,
        cls=(NavT.primary, "space-y-4 p-4 w-full md:w-full")
    )

In [71]:
#| export
@rt
def index():return getting_started()

In [53]:
#| export
serve()